프로젝트 : 다양한 OCR모델 비교하기
======

Google OCR API, keras-ocr, Tesseract 3가지의 OCR 모델을 비교해 봅니다. 
     
1. 검증용 데이터셋 준비
2. Google OCR API, keras-ocr, Tesseract로 테스트 진행
3. 테스트 결과 정리
4. 결과 분석과 결론 제시
    
-21.12.02-